I have already downloaded over 6000 fanfiction works from https://archiveofourown.org/media using a separate python script (this script is included separately in the upload and is linked to in my markdown file), and I'm working on downloading more, but I will process that data here using the python library BeautifulSoup. Data collection with scrapy takes time, and I'll update this as soon as I have my full dataset.

In [1]:
from bs4 import BeautifulSoup
import os
import glob
import lxml

file_location = os.path.join('C:/Users/rsbru/OneDrive/Documents/data_science/Fanfiction-Classification-Analysis/data/', '*.html')
filenames = glob.glob(file_location)
files = {}
for f in filenames:
    with open(f, encoding = 'utf8') as fp:
        soup = BeautifulSoup(fp, 'lxml')
        file_name = f[87:]
        files[file_name] = soup

Now that I have my files read into python as searchable html files, I can use BeautifulSoup's search operations to pull out the parts of the document that are relevant to my project (namely the text itself and the tags attached to the work).

In [2]:
import pandas as pd
file_names = [f[87:] for f in filenames]
ratings = {}
warnings = {}
categories = {}
fandoms = {}
relationships = {}
characters = {}          
additional = {}
titles = {}
chapters = {}
texts = {}
for fn in file_names:
    if(files[fn].find('dd', class_="rating tags") is not None):
        rats = []
        for child in files[fn].find('dd', class_="rating tags").find_all('a'):
            for string in child.stripped_strings:
                rats.append(string)
        ratings[fn] = ', '.join(rats)
    if(files[fn].find('dd', class_="warning tags") is not None):
        warns = []
        for child in files[fn].find('dd', class_="warning tags").find_all('a'):
            for string in child.stripped_strings:
                warns.append(string)
        warnings[fn] = ', '.join(warns)
    if(files[fn].find('dd', class_="category tags") is not None):
        cats = []
        for child in files[fn].find('dd', class_="category tags").find_all('a'):
            for string in child.stripped_strings:
                cats.append(string)
        categories[fn] = ', '.join(cats)
    if(files[fn].find('dd', class_="fandom tags") is not None):
        fans = []
        for child in files[fn].find('dd', class_="fandom tags").find_all('a'):
            for string in child.stripped_strings:
                fans.append(string)
        fandoms[fn] = ', '.join(fans)
    if(files[fn].find('dd', class_="relationship tags") is not None):
        rels = []
        for child in files[fn].find('dd', class_="relationship tags").find_all('a'):
            for string in child.stripped_strings:
                rels.append(string)
        relationships[fn] = ', '.join(rels)
    if(files[fn].find('dd', class_="character tags") is not None):
        chars = []
        for child in files[fn].find('dd', class_="character tags").find_all('a'):
            for string in child.stripped_strings:
                chars.append(string)
        characters[fn] = ', '.join(chars)
    if(files[fn].find('dd', class_="freeform tags") is not None):
        adds = []
        for child in files[fn].find('dd', class_="freeform tags").find_all('a'):
            for string in child.stripped_strings:
                adds.append(string)
        additional[fn] = ', '.join(adds)
    if(files[fn].find(class_="title heading") is not None):
        for child in files[fn].find(class_="title heading").find_all('a'):
            if(child.string is not None):
                titles[fn] = child.string
    if(files[fn].find(class_="chapter preface group") is not None):
        for child in files[fn].find(class_="chapter preface group").find_all('a'):
            if(child.string is not None):
                chapters[fn] = child.string
    tex = files[fn].find_all('p')
    pars = []
    for text in tex:
        for string in text.stripped_strings:
            pars.append(string)
    texts[fn] = '\n'.join(pars)

I will now organize the data I've pulled out of my raw files into a DataFrame using pandas.

In [3]:
fanfics = pd.DataFrame({'Filename': file_names})
fanfics['Rating'] = fanfics['Filename'].map(ratings)
fanfics['Warning'] = fanfics['Filename'].map(warnings)
fanfics['Category'] = fanfics['Filename'].map(categories)
fanfics['Fandom'] = fanfics['Filename'].map(fandoms)
fanfics['Relationships'] = fanfics['Filename'].map(relationships)
fanfics['Characters'] = fanfics['Filename'].map(characters)
fanfics['Additional'] = fanfics['Filename'].map(additional)
fanfics['Title'] = fanfics['Filename'].map(titles)
fanfics['Chapter'] = fanfics['Filename'].map(chapters)
fanfics['Text'] = fanfics['Filename'].map(texts)

In [4]:
fanfics.head()

,Filename,Rating,Warning,Category,Fandom,Relationships,Characters,Additional,Title,Chapter,Text
0,1005380.html,Explicit,Creator Chose Not To Use Archive Warnings,M/M,Harry Potter - J. K. Rowling,"Sirius Black/Remus Lupin, James/Lily, Peter Pe...","Remus Lupin, Sirius Black, James Potter, Lily ...","Humor, Angst, First War with Voldemort",NaN,Chapter 2,While we've done our best to make the core fun...
1,10057010.html,Mature,"No Archive Warnings Apply, Major Character Dea...",M/M,Harry Potter - J. K. Rowling,"Sirius Black/Remus Lupin, Sirius Black & Remus...","Remus Lupin, Sirius Black, James Potter, Lily ...","Marauders' Era, Marauders, Marauders Friendshi...",NaN,Chapter 3,While we've done our best to make the core fun...
2,10184531.html,Teen And Up Audiences,No Archive Warnings Apply,Gen,The Silmarillion and other histories of Middle...,"Morgoth Bauglir | Melkor & Sauron | Mairon, El...","Melian (Tolkien), Thranduil (Tolkien), Thrandu...","Epic Friendship, Deep enmity, Sorrow and Grief...",NaN,Chapter 101,While we've done our best to make the core fun...
3,1018692.html,Explicit,"Graphic Depictions Of Violence, Underage","Multi, F/M, M/M","Homestuck, MS Paint Adventures","Sollux Captor/Aradia Megido, Gamzee Makara/Tav...","Karkat Vantas, Gamzee Makara, Sollux Captor, J...","Troll Cameos, Explicit Language, Underaged Dru...",NaN,Chapter 1,While we've done our best to make the core fun...
4,1020833.html,Teen And Up Audiences,Creator Chose Not To Use Archive Warnings,NaN,逆転裁判 | Gyakuten Saiban | Ace Attorney,Mitsurugi Reiji/Naruhodou Ryuuichi | Miles Edg...,"Mitsurugi Reiji | Miles Edgeworth, Naruhodou R...","Fluff, Comedy, Dual Destinies Era, Phoenix Wri...",NaN,Chapter 34,While we've done our best to make the core fun...


Just for fun here (and to make sure my DataFrame is functional), let's look at the most common fandoms in my (limited) dataset.

In [5]:
fanfics.Fandom.value_counts()

Harry Potter - J. K. Rowling                                                                                                                    445
僕のヒーローアカデミア | Boku no Hero Academia | My Hero Academia                                                                                          287
Naruto                                                                                                                                          124
Encanto (2021)                                                                                                                                  116
Five Nights at Freddy's                                                                                                                         113
                                                                                                                                               ... 
The Way to Protect the Female Lead's Older Brother, 여주인공의 오빠를 지키는 방법 - 킨 | The Way to Protect the Female Lead's 

I'm not entirely finished collecting my data, and I still need to do a little cleaning of my DataFrame before I can fully work with it for analysis, but again I will update this script as I work on that.